In [2]:
#####################################################################
# Created by : Charlotte Menou and Ange Leyrit                      #
# version : 7                                                       #
# Un prototype d'un space invaders avec le joueur qui peut tirer    #
# et se déplacer, les Aliens sont détruit par les balles et se      #
# déplacent eux aussi.                                              #
# Ecran de VICTOIRE et DEFAITE avec affichage du score de la partie #
# en terminée.                                                      #                                                                  
#####################################################################

In [8]:
try:  
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox
    
#classe de gestion d'un alien, de récupération de son image, de son mouvement et qu'il soit en vie ou non   
class Alien(object):

    alive_image=None
    
    def __init__(self):
        self.id = None
        self.alive = True
    
    @classmethod #méthode de classe de récupération de l'image d'un alien
    def get_alive_image(cls):
        if cls.alive_image==None:
            cls.alive_image=tk.PhotoImage(file='alien.gif')    
        return cls.alive_image
            
    def touched_by(self, canvas):#méthode permettant de gérer la "mort" d'un alien
        self.alive=False             
        canvas.delete(self.id)       
        
    def install_in(self, canvas, x, y):#méthode de classe d'installation d'un alien
        self.id=canvas.create_image(x,y,image=Alien.get_alive_image())    
        
    def move_in(self, canvas, dx, dy): #méthode de classe de déplacement d'un alien
        if self.alive==True:           
            canvas.move(self.id,dx,dy)     


#classe de gestion de la flotte, de ses mouvement, et de la collision Alien/Bullet
class Fleet(object):
    def __init__(self):
        self.score=0
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 20
        self.alien_x_delta = 10
        self.alien_y_delta =25
        self.fleet_size = self.aliens_lines*self.aliens_columns
        self.aliens_fleet = [] 
        self.nb_aliens_restants=50
        self.nb_deplacements=0
        self.bas_de_la_matrice=350

    def install_in(self, canvas):#méthode d'installation de la flotte d'aliens
        x=50
        y=50
        for j in range(self.aliens_lines):
            for i in range(self.aliens_columns):
                alien=Alien()
                alien.install_in(canvas,x,y)        
                self.aliens_fleet.append(alien)
                x=x+80         #changement de colonne
            x=x-80*self.aliens_columns
            y=y+70              #changement de ligne
            
    def move_in(self, canvas):#méthode de gestion du mouvement de la flotte
        if self.nb_deplacements==19:           #si la flotte a effectué 19 déplacement dans un sens, elle est arrivée à un bord de l'écran
            self.alien_x_delta = -self.alien_x_delta     #on fait avancer la flotte dans l'autre sens
            for i in self.aliens_fleet:    
                i.move_in(canvas,0,self.alien_y_delta)     #on fait descendre la flotte d'aliens vers le bas de l'écran
            self.bas_de_la_matrice+=self.alien_y_delta     #on stock la valeur du bas de la flotte pour le game over
            self.nb_deplacements=0          
        for i in self.aliens_fleet:
            i.move_in(canvas,self.alien_x_delta,0)
    
        
    def manage_touched_aliens_by(self,canvas,defender): #méthode de gestions des colissions alien/bullet
        for alien in self.aliens_fleet.copy():        
            if alien.alive:                       
                x1, y1, x2, y2=canvas.bbox(alien.id)       
                touched=canvas.find_overlapping(x1, y1, x2, y2)     #on cherche si quelque chose touche l'alien
                if(len(touched)>1):       #si la longeur de "touched" est >1, cela signifie que quelque chose touche l'alien
                    alien.touched_by(canvas)        
                    for bullet in defender.fired_bullets:       
                        if bullet.id==touched[1]:             #boucle permettant de trouver si ce qui touche l'alien est un bullet afin de lancer sa suppresion et celle de l'alien
                            self.score=self.score+1
                            defender.fired_bullets.remove(bullet)     
                            canvas.delete(bullet.id)              
                            defender.max_fired_bullets=defender.max_fired_bullets+1       #on recharge d'un coup
                            self.nb_aliens_restants-=1            #on diminue le nombre d'aliens restants
                
                    
                    
#classe de gestion du Defender(joueur), pour gérer ses mouvement, le tir des bullet
class Defender(object):
    def __init__(self):
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets = 8     
        self.fired_bullets = []          

    def install_in(self, canvas): #méthode d'installation du defender
        x, y= 25, 660
        self.id = canvas.create_rectangle(x,y,x+self.width,y+self.height,fill="white")      
        
    def move_in(self,canvas, dx): #méthode de mouvement du defender
        canvas.move(self.id, dx, 0)                  

    def fire(self, canvas): #méthode de création d'une bullet, d'ajout à la liste de bullet tirer et d'installation dans le canvas
        bullet=Bullet(self)                               
        self.fired_bullets.append(bullet)                 
        bullet.install_in(canvas)                         
        
#classe de gestion des bullet, pour gérer leur mouvement vers le haut et leur affichage au dessus du defender       
class Bullet(object):
    def __init__(self,shooter):
        self.radius = 5
        self.color = "red"
        self.speed = 20
        self.id = None
        self.shooter = shooter

    def install_in(self, canvas):#méthode de création d'une bullet au dessus du defender dans le canvas
        x1, y1, x2, y2 = canvas.coords(self.shooter.id)                
        self.id = canvas.create_oval(x1+5,y1-20,x1+5+2*self.radius,y1-20+2*self.radius,fill=self.color) 

    def move_in(self, canvas): #méthode du mouvement d'une bullet vers le haut de l'écran
            canvas.move(self.id, 0, -self.speed)       
        
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.player="Ange_or_Charlotte"
        self.height =700
        self.width=1000
        self.canvas = tk.Canvas(self.frame, width=self.width, height=self.height, background='black')
        self.defender.install_in(self.canvas)     #installation du defender dans le canvas
        self.canvas.pack()
        self.fleet.install_in(self.canvas)        #installation de la flotte d'aliens dans le canvas

    @classmethod #méthode de classe, pour récupérer l'image de fin de jeux perdu
    def get_gameover(cls):
        cls.gameover_image=tk.PhotoImage(file='gameover.gif')    
        return cls.gameover_image
    
    @classmethod #méthode de classe, pour récupérer l'image de fin de jeux gagner
    def get_winner(cls):
        cls.winner_image=tk.PhotoImage(file='winner1.gif')   
        return cls.winner_image
    
    def animation(self): 
        self.frame.winfo_toplevel().bind("<Key>", self.keypress)
        self.move_bullets()                                               #mouvement des bullets
        self.move_aliens_fleet()                                          #mouvement de la flotte d'aliens
        self.fleet.manage_touched_aliens_by(self.canvas,self.defender)    #gestion de la collision d'aliens avec des bullets
        if self.fleet.bas_de_la_matrice>=665 and self.fleet.nb_aliens_restants>0:    #si la fotte arrive plus bas que le defender et qu'il reste des aliens
            self.game_over()                                              #le jeu est perdu
        elif self.fleet.nb_aliens_restants==0:                            #si il n'y a plus d'aliens en vie 
            self.winner()                                                 #le jeu est gagné
        self.canvas.after(300, self.animation)         
        
    def keypress(self, event): #méthode pour gestion des touches du clavier pour defender
        x = 0
        x1, y1, x2, y2 = self.canvas.coords(self.defender.id)
        if event.keysym == 'Left' and x2>50:          #si on appuie sur la fleche gauche du clavier et que l'ordonnée du defender est >50
            x = -self.defender.move_delta             #on bouge le defender vers la gauche
        elif event.keysym == 'Right' and x2<965:      #si on appuie sur la fleche droite du clavier et que l'ordonnée du defender est <965
            x = self.defender.move_delta              #on bouge le defender vers la droite
        self.defender.move_in(self.canvas, x)
        if event.keysym == 'space' and self.defender.max_fired_bullets>0:          #si on appuie sur la touche espace du clavier et que le defender peut encore tirer des bullets
            self.defender.max_fired_bullets=self.defender.max_fired_bullets-1       #on retire un coup au defender
            self.defender.fire(self.canvas)             #le defender lance un bullet
            
             
    def start_animation(self):
        self.canvas.after(10, self.animation())
        
    def move_bullets(self):#méthode permettant de limiter le nombre de bullet tirer à 8
        for bullet in self.defender.fired_bullets:            
            bullet.move_in(self.canvas)                        
            x1, y1, x2, y2 = self.canvas.coords(bullet.id)     #on récupère les coordonnées du bullet
            if (y2<0 and self.defender.max_fired_bullets<8) :                #si son ordonnée est <0 et qu'il y a moins de 8 bullets à l'écran
                    self.defender.fired_bullets.remove(bullet)             #on suprime le bullet de la liste
                    self.defender.max_fired_bullets=self.defender.max_fired_bullets+1         #on recharge un coup
             
    def move_aliens_fleet(self):
        self.fleet.move_in(self.canvas)
        self.fleet.nb_deplacements+=1
        
    def game_over(self):#méthode d'affichage de l'écran de défaite et du score
        self.canvas.create_rectangle(0,0,1000,700,fill="black")     #on recouvre le canvas d'un grand restangle noir
        self.canvas.create_image(500,350,image=Game.get_gameover())       #création de l'image "game over"
        self.canvas.create_text(500,450, text ='JOUEUR : '+ str(self.player)+'     ' +'SCORE : '+str(self.fleet.score) , fill = 'white', font = (60))  
    
    def winner(self):#méthode d'affichage de l'écran de victoire et du score 
        self.canvas.create_rectangle(0,0,1000,700,fill="black")        
        self.canvas.create_image(500,300,image=Game.get_winner())        #création de l'image "you win"
        self.canvas.create_text(500,450, text = 'JOUEUR : '+ str(self.player)+'     ' +'SCORE : '+str(self.fleet.score), fill = 'white', font = (60))  
        
        
class SpaceInvaders(object):

    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both", expand=True)
        self.game=Game(self.frame)
    def play(self):
        self.game.start_animation()
        self.root.mainloop()
                        
                        
SpaceInvaders().play()